In [ ]:
!pip install torchtext --upgrade
!pip install transformers
!pip install pytorch_lightning

Requirement already up-to-date: torchtext in /usr/local/lib/python3.7/dist-packages (0.9.1)
     |████████████████████████████████| 2.3MB 9.8MB/s 
     |████████████████████████████████| 901kB 53.3MB/s 
     |████████████████████████████████| 3.3MB 51.6MB/s 
     |████████████████████████████████| 808kB 7.5MB/s 
     |████████████████████████████████| 112kB 54.6MB/s 
     |████████████████████████████████| 276kB 47.4MB/s 
     |████████████████████████████████| 10.6MB 51.0MB/s 
     |████████████████████████████████| 829kB 44.5MB/s 
     |████████████████████████████████| 645kB 39.4MB/s 
     |████████████████████████████████| 1.3MB 45.2MB/s 
     |████████████████████████████████| 143kB 55.3MB/s 
     |████████████████████████████████| 296kB 49.5MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=6781d5b6671781faf0273925a3982fa990dfdd4e8bbb90e4a406c4f85575e120
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886

In [ ]:
!git clone https://github.com/SapienzaNLP/nlp2021-hw2

Cloning into 'nlp2021-hw2'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 32 (delta 10), reused 18 (delta 3), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [ ]:
import os

data_folder = os.sep.join(["nlp2021-hw2", "data"])
training_file = [os.sep.join([data_folder, "restaurants_train.json"]), os.sep.join([data_folder, "laptops_train.json"])]
dev_file = [os.sep.join([data_folder, "restaurants_dev.json"]),os.sep.join([data_folder, "laptops_dev.json"])]

In [ ]:
# here go all the imports
import torch
from torch import nn
from torch.utils.data import Dataset
from torchtext import data
from torchtext.vocab import Vectors

from transformers import BertTokenizer, BertModel


from pprint import pprint
from tqdm import tqdm
from torchtext.vocab import Vocab
from collections import Counter, defaultdict
import random
import numpy as np

from typing import *

import json
import re

import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torch.optim as optim

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report 


Set up the seed and deterministic algorithms in order to have reproducible results

In [ ]:
SEED = 96

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Load Data

In [ ]:
class MyDataset(Dataset):

    def __init__(self, 
                 input_files, 
                 device="cpu"):
        """
        Args:
            input_file (list of strings or paths): each element is a path to a dataset to be loaded.
            device (string): device where to put tensors (cpu or cuda).
        """
        # open files and load them to list of dictionaries
        self.input_files = input_files
        with open(input_files[0]) as fin:
            data0 = json.load(fin)
        with open(input_files[1]) as fin:
            data1 = json.load(fin)
        
        # aggregate the data
        self.data = data0 + data1
        
        # count the total number of targets
        self.num_targets = 0
        for entry in self.data:
            targets_pos = [target[0] for target in entry["targets"]] 
            self.num_targets += len(targets_pos)

        # associate lables to OH lists
        self.classification_labels = {
            "positive" : [1,0,0,0],
            "negative" : [0,1,0,0],
            "neutral" : [0,0,1,0],
            "conflict" : [0,0,0,1],
        }

        self.device = device

        # count for each class the number of entries
        self.classes_number = {"negative":0, "positive":0, "conflict":0, "neutral":0}

        self.__init_data__()


    def __init_data__(self):
        self.encoded_data = list()
        
        # for each phrase
        for i in range(len(self.data)):
            # get the text and lowerize it
            text = self.data[i]["text"].lower()
            
            # for each target
            for target in self.data[i]["targets"]:
                
                # replace the whole target with a special token
                text_with_target = text[:target[0][0]] + " <target-term> " + text[target[0][1]:] 
                
                # check the label
                label = target[2] 
                assert label in ["negative", "positive", "conflict", "neutral"]
                
                # update the counter of the obtained class
                self.classes_number[label] += 1

                # associate the label to a OH list
                label = self.classification_labels[label]

                # pass the label to a tensor 
                label = torch.Tensor(label).float().to(self.device)
                self.encoded_data.append({"inputs":text_with_target, 
                                          "outputs":label})

    def get_classes_number(self):
        return self.classes_number

    def __len__(self):
        return self.num_targets

    def __getitem__(self, idx):
        return self.encoded_data[idx]
    

In [ ]:
dataset = MyDataset(training_file)
classes_number = dataset.get_classes_number()

In [ ]:
# build the weights for the loss, so that classes with a lower number of entries
# will have higher cost
total_entries = classes_number["positive"] + classes_number["negative"] + classes_number["neutral"] + classes_number["conflict"]
weights = [(total_entries-classes_number["positive"])/total_entries, (total_entries-classes_number["negative"])/total_entries, (total_entries-classes_number["neutral"])/total_entries, (total_entries-classes_number["conflict"])/total_entries]

#### Define the model

In [ ]:
class MyModel(nn.Module):
    
    """
        hparams: dict with dropout and num_classes informations
        tokenizer: the tokenizer used to tokenize the sentences
        
    """

    def __init__(self, hparams, tokenizer):
        super(MyModel, self).__init__()

        # download the bert pretrained model        
        self.bert = BertModel.from_pretrained('bert-large-uncased', output_hidden_states=True)
        
        # resize the embeddings since I added a new token (<target-term>)
        self.bert.resize_token_embeddings(len(tokenizer))

        #initialize dropout and two FC layers
        self.dropout = nn.Dropout(hparams.dropout)

        # 1024 is the output size for bert large
        # 4 * 1024 since I am going to concatenate the last 4 layers
        self.lin1 = torch.nn.Linear(4*1024, hparams.num_classes) 

    def forward(self, input, indices):

        bert_out = self.bert(**input)

        # get the output of the last 4 layers relative to the <target-term> word_piece
        last = bert_out.hidden_states[-1][range(input["input_ids"].shape[0]), indices, :]
        last_but_one = bert_out.hidden_states[-2][range(input["input_ids"].shape[0]), indices, :]
        last_but_two = bert_out.hidden_states[-3][range(input["input_ids"].shape[0]), indices, :]
        last_but_three = bert_out.hidden_states[-4][range(input["input_ids"].shape[0]), indices, :]

        # stack the outputs
        bert_out = torch.hstack((last, last_but_one, last_but_two, last_but_three))
        last, last_but_one, last_but_two, last_but_three = None, None, None, None
        
        # dropout
        bert_out = self.dropout(bert_out)

        # FC
        out = self.lin1(bert_out)

        return out

        

#### Pytorch Lightning Module (Train, Dev and Test code)

In [ ]:
class MyLightningModule(pl.LightningModule):

    
    def __init__(self, hparams, tokenizer, *args, **kwargs):
        super(MyLightningModule, self).__init__(*args, **kwargs)
    
        # save hparams
        self.save_hyperparameters(hparams)
    
        # initialize the loss function as a weighted CrossEntropyLoss for multiclass classification 
        self.loss_function = nn.CrossEntropyLoss(weight=self.hparams.weights)
        
        # initialize the model we want to train
        self.model = MyModel(self.hparams, tokenizer)

        self.logits = []
        self.labels_indices = []
        self.arg_maxs = []
    
    # This performs a forward pass of the model
    # returns the predicted logits
    """
        x is a dictionary containing the inputs_ids and the attention mask: the parameters of bert
        indices is a list containing the index of the target-term for each sentence
    """
    
    def forward(self, x, indices):
        logits = self.model(x, indices)

        return logits

    """
        batch -> dict containing 
                inputs: tokenized sentences (sequence of integers) padded
                        and attention mask
                outputs: correct labelling
                indices: position of the <target-term> wordpiece in each sentence
    """
    def training_step(self, batch, batch_nb):

        inputs = batch['inputs'] # bert input
        labels = batch['outputs'] # the true labels
        indices = batch['indices'] # position of the target-term word piece

        # get the predicted logits
        logits = self.forward(inputs, indices)
        
        # apply softmax in order to obtain a probability distribution over the labels
        preds = nn.Softmax(-1)(logits)
        
        # get the predicted label and the correct one
        arg_max = torch.argmax(preds, dim=-1)
        labels_indices = torch.argmax(labels, dim=-1)
      
        # compute the loss
        loss = self.loss_function(logits, labels_indices)
        
        self.log('train_loss', loss, prog_bar=True)
        
        return loss
    

    # validation step -> model in eval state 
    """
        batch -> dict containing 
                inputs: tokenized sentences (sequence of integers) padded
                outputs: correct labelling
                indices: position of the <target-term> word piece in each sentence
    """
    def validation_step(self, batch, batch_nb):
        inputs = batch['inputs']
        labels = batch['outputs']
        indices = batch['indices']
        

        # get the predicted logits
        logits = self.forward(inputs, indices)
        
        # apply softmax in order to obtain a probability distribution over the labels
        preds = nn.Softmax(-1)(logits)
        
        # get the predicted label and the correct one
        arg_max = torch.argmax(preds, dim=-1)
        labels_indices = torch.argmax(labels, dim=-1)
        
        # compute the loss and the f1 score
        valid_loss = self.loss_function(logits, labels_indices)
        sample_f1 = f1_score(labels_indices.detach().cpu(), arg_max.detach().cpu(), average="macro")
          
        self.log('valid_loss', valid_loss, prog_bar=True)
        self.log('valid_f1', sample_f1, prog_bar=True)


    # test step -> model in eval state
    """
        batch -> dict containing 
                inputs: tokenized sentences (sequence of integers) padded
                outputs: correct labelling
                indices: position of the <target-term> word piece in each sentence
    """
    def test_step(self, batch, batch_nb):
        inputs = batch['inputs']
        labels = batch['outputs']
        indices = batch['indices']

        # get the predicted logits
        logits = self.forward(inputs, indices)
       
        # apply softmax in order to obtain a probability distribution over the labels
        preds = nn.Softmax(-1)(logits)
        
        # get the predicted label and the correct one
        arg_max = torch.argmax(preds, dim=-1)
        labels_indices = torch.argmax(labels, dim=-1)

        self.logits += logits.detach().cpu().tolist()
        self.labels_indices += labels_indices.detach().cpu().tolist()
        self.arg_maxs += arg_max.detach().cpu().tolist()

        # compute the loss and the f1 score
        #test_loss = self.loss_function(self.logits, self.labels_indices)
        sample_f1 = f1_score(self.labels_indices, self.arg_maxs, average="macro")
        f1_per_class = f1_score(self.labels_indices, self.arg_maxs, average=None)


        #self.log('test_loss', test_loss, prog_bar=True)
        self.log('test_f1', sample_f1, prog_bar=True)
        self.log('f1 positive', f1_per_class[0], prog_bar=True)
        self.log('f1 negative', f1_per_class[1], prog_bar=True)
        self.log('f1 neutal', f1_per_class[2], prog_bar=True)
        self.log('f1 conflict', f1_per_class[3], prog_bar=True)

        print(classification_report(self.labels_indices, self.arg_maxs))

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.000_01)


In [ ]:
# get the bert tokenizer and add a new special token (for the target term)
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', additional_special_tokens=["<target-term>"])

# get the id of the target term
target_id = tokenizer("<target-term>", return_tensors="pt")["input_ids"][0][1]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
""" 
data is a list of dictionaries containing each 
       inputs : the sentences where the aspect terms has been replaced with the special token
       outputs: the true labels
"""
def collate_fn(data):
    # build a list of sentences
    X = [entry["inputs"] for entry in data]
    
    # tokenize each sentence and pad them 
    X = tokenizer(X, return_tensors="pt", padding=True)
    
    # stack the true labels
    y = torch.vstack([entry["outputs"] for entry in data]) 
    
    # get theindex for each sentence where the target term is
    index = [(X["input_ids"][i] == target_id).nonzero(as_tuple=True)[0] for i in range(X["input_ids"].shape[0])]

    return {"inputs":X, "outputs":y, "indices":index}

In [ ]:
class MyLightningDataModule(pl.LightningDataModule):
    """
        training_file: list of the two training files paths
        dev_file: list of the two dev files paths
        collate_fn: the collete fun
    """
    def __init__(self, training_file, dev_file, collate_fn, device="cpu"):
        super().__init__()
        self.training_file = training_file
        self.dev_file = dev_file
        self.collate_fn = collate_fn
        self.device = device

    def setup(self, stage=None):
      # initialize the two datasets
      self.trainingset = MyDataset(self.training_file, device=self.device)
      self.devset = MyDataset(self.dev_file, device=self.device)

    def train_dataloader(self):
        return DataLoader(self.trainingset, batch_size=32, collate_fn = self.collate_fn, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.devset, batch_size=256, collate_fn = self.collate_fn)
    
    def test_dataloader(self):
        return DataLoader(self.devset, batch_size=len(self.devset)//3, collate_fn = self.collate_fn)
    

#### Define Hyperparams and start Training

In [ ]:
# params of the model
hparams = { 
    'num_classes': 4, 
    'dropout': 0.3,
    'weights': torch.tensor(weights)
    }

In [ ]:
# define early stopping and checkpoint callbacks
early_stop_callback = EarlyStopping(
   monitor="valid_loss",
   min_delta=0.00,
   patience=3,
   verbose=False,
   mode="min"
)


checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best_model",
    monitor = "valid_loss",
    mode = "min",
    save_last=True
)
# initialize the data module and the trainer
data_module = MyLightningDataModule(training_file, dev_file, collate_fn, device=device)
trainer = pl.Trainer(val_check_interval=1.0, max_epochs=20, gpus=1 if device!=torch.device('cpu') else None, callbacks=[early_stop_callback, checkpoint_callback])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
# initialize the model and train it
model = MyLightningModule(hparams, tokenizer=tokenizer)
trainer.fit(model, datamodule=data_module)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | loss_func

#### Model Evaluation

In [ ]:
# test the overfitted model
trainer.test(model, test_dataloaders=data_module.test_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


              precision    recall  f1-score   support

           0       0.87      0.89      0.88       197
           1       0.74      0.65      0.69        83
           2       0.58      0.61      0.59        74
           3       0.30      0.30      0.30        10

    accuracy                           0.76       364
   macro avg       0.62      0.61      0.62       364
weighted avg       0.76      0.76      0.76       364

              precision    recall  f1-score   support

           0       0.86      0.90      0.88       391
           1       0.76      0.66      0.71       178
           2       0.52      0.53      0.52       139
           3       0.35      0.40      0.37        20

    accuracy                           0.76       728
   macro avg       0.62      0.62      0.62       728
weighted avg       0.76      0.76      0.76       728

              precision    recall  f1-score   support

           0       0.86      0.88      0.87       546
           1       0.

[{'f1 conflict': 0.35320842266082764,
  'f1 negative': 0.7296035289764404,
  'f1 neutal': 0.5812013149261475,
  'f1 positive': 0.875967264175415,
  'test_f1': 0.63499516248703}]

In [ ]:
# load the best model (the one with lower loss)
chk_path = "checkpoints/best_model.ckpt"
model = MyLightningModule.load_from_checkpoint(chk_path, hparams, tokenizer=tokenizer)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# test the best model
trainer.test(model, test_dataloaders=data_module.test_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


              precision    recall  f1-score   support

           0       0.88      0.85      0.87       197
           1       0.71      0.73      0.72        83
           2       0.51      0.57      0.54        74
           3       0.00      0.00      0.00        10

    accuracy                           0.74       364
   macro avg       0.52      0.54      0.53       364
weighted avg       0.74      0.74      0.74       364

              precision    recall  f1-score   support

           0       0.87      0.87      0.87       391
           1       0.73      0.74      0.73       178
           2       0.47      0.52      0.49       139
           3       0.00      0.00      0.00        20

    accuracy                           0.74       728
   macro avg       0.52      0.53      0.52       728
weighted avg       0.73      0.74      0.74       728

              precision    recall  f1-score   support

           0       0.88      0.84      0.86       546
           1       0.

[{'f1 conflict': 0.0,
  'f1 negative': 0.7443082332611084,
  'f1 neutal': 0.5334169864654541,
  'f1 positive': 0.8620957136154175,
  'test_f1': 0.5349552631378174}]

Min Loss:
              
                precision    recall  f1-score   support

           0       0.88      0.84      0.86       546
           1       0.76      0.77      0.76       307
           2       0.51      0.61      0.55       216
           3       0.00      0.00      0.00        25

     acc                               0.75      1094
     M avg         0.54      0.55      0.54      1094
     W avg         0.75      0.75      0.75      1094

Last Model:
              
                precision    recall  f1-score   support

           0       0.86      0.88      0.87       546
           1       0.81      0.71      0.76       307
           2       0.57      0.64      0.60       216
           3       0.34      0.40      0.37        25

     acc                               0.77      1094
     M avg         0.65      0.66      0.65      1094
     W avg         0.78      0.77      0.78      1094

In [ ]:
# load TensorBoard 
%load_ext tensorboard

In [ ]:
%tensorboard --logdir lightning_logs/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! ls checkpoints

best_model.ckpt  last.ckpt


In [ ]:
! cp checkpoints/last.ckpt drive/MyDrive/